##Notebook to process MODIS Data and calculate NDVI Value##

In [1]:
import glob
from pyhdf.SD import SD, SDC
import numpy as np
import math
from __future__ import print_function
import matplotlib
# matplotlib.use('TkAgg') # use to make your graph pop out of notebook
import pylab
import matplotlib.pyplot as plt
import datetime
import subprocess
from numpy import genfromtxt
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
import time
import numexpr as ne # parallel computation
import bottleneck as bn

In [4]:
#Get path to single hdf file
file = SD('/panfs/roc/groups/6/kumarv/tayal007/MODIS_TILES/MODIS_h08v05/MOD09A1.A2000217.h08v05.006.2015138053132.hdf', SDC.READ)
print (file.info())

(13, 6)


In [5]:
datasets_dic = file.datasets()
for idx,sds in enumerate(datasets_dic.keys()):
    print (idx, sds)

0 sur_refl_b01
1 sur_refl_b02
2 sur_refl_b03
3 sur_refl_b04
4 sur_refl_b05
5 sur_refl_b06
6 sur_refl_b07
7 sur_refl_qc_500m
8 sur_refl_szen
9 sur_refl_vzen
10 sur_refl_raz
11 sur_refl_state_500m
12 sur_refl_day_of_year


In [6]:
#sds_obj = file.select('Land_Cover_Type_1') 
sds_obj = file.select('sur_refl_b01')  # select sds
data = np.array(sds_obj.get()) # get sds data
print (data.shape)
print (data)
#plt.imshow(data)
#plt.show()

(2400, 2400)
[[1255 1437 1753 ... 1545 1778 1778]
 [1704  938  938 ... 1778 1778 1960]
 [ 946  938  618 ... 1919 1919 2041]
 ...
 [1348 1324 1324 ... 1179 1284 1378]
 [1466 1358 1358 ... 1230 1345 1345]
 [1369 1348 1348 ... 1292 1292 1219]]


In [7]:
import pprint
pprint.pprint( (sds_obj.attributes()))

{'_FillValue': -28672,
 'add_offset': 0.0,
 'add_offset_err': 0.0,
 'calibrated_nt': 5,
 'long_name': 'Surface_reflectance_for_band_1',
 'scale_factor': 0.0001,
 'scale_factor_err': 0.0,
 'units': 'reflectance',
 'valid_range': [-100, 16000]}


In [2]:
def Timeseriesmatrix(directorypath, sorteddates):
    NDVI_CON = np.zeros(shape=(2400,2400,len(sorteddates)),dtype=np.float32)
    i = 0
    for k, v in sorted(sorteddates.items()):
        start = time.time()
        print (i)
        matching = [s for s in directorypath if v in s]
        print (matching[0])
        file = SD(matching[0], SDC.READ)
        sds_obj01 = file.select('sur_refl_b01')  # select sds Red
        sds_obj02 = file.select('sur_refl_b02')  # select  sds NIR
        sds_obj03 = file.select('sur_refl_state_500m') # select quality sds
        Red = np.array(sds_obj01.get()) # get sds data
        NIR = np.array(sds_obj02.get())
        Quality = np.array(sds_obj03.get())
        assert (Red.shape == NIR.shape == Quality.shape);
        Quality = Quality & 3
        NDVI_CON[:,:,i] = ne.evaluate('((NIR - Red))/((NIR + Red))').astype(np.float32);
        NDVI_CON[:,:,i][np.logical_or.reduce((Quality!= 0,NIR==-28672,Red==-28672))] = np.nan
        end = time.time()
        print ((end - start))
        i = i + 1
    return NDVI_CON

def sortedtimestamp(directorypath):
   timestamps = dict()
   for path in directorypath:
        timestamps[datetime.datetime.strptime(path.split('.')[1][3:], '%y%j').date()] = path.split('.')[1]
   return timestamps;

In [3]:
directorypath = glob.glob("/home/kumarv/tayal007/MODIS_TILES/MODIS_h20v05/*.hdf");
# print (directorypath)
sorteddates = sortedtimestamp(directorypath)
NDVISTACK = Timeseriesmatrix(directorypath, sorteddates);
print ("Final NDVI shape is " + str(NDVISTACK.shape));
#np.save('/home/kumarv/tayal007/MODIS_TILES/h21v07', NDVISTACK)

In [ ]:
print (NDVISTACK.shape)
print (np.count_nonzero(np.isnan(NDVISTACK)))
print (np.count_nonzero(np.isinf(NDVISTACK)))

In [ ]:
# check range NDVI values, excluding NaN
np.nanmin(NDVISTACK), np.nanmax(NDVISTACK)

In [ ]:
def percentage(part, shape):
    whole = shape[0] * shape[1] * shape[2]
    return 100 * float(part)/float(whole)

def cleanNDVI(NDVISTACK):
	print ("Missing value in raw data is ", np.count_nonzero(np.isnan(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isnan(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Infinity value in raw data is ", np.count_nonzero(np.isinf(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isinf(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Greater than 1 is ", np.count_nonzero(NDVISTACK > 1),'which is ' ,percentage(np.count_nonzero(NDVISTACK > 1),NDVISTACK.shape),'%')
	print ("Less than -1 is ", np.count_nonzero(NDVISTACK < -1),'which is ' ,percentage(np.count_nonzero(NDVISTACK < -1),NDVISTACK.shape),'%')
	
	CLEANED_DATA = np.copy(NDVISTACK)
	CLEANED_DATA[NDVISTACK == inf] = np.nan
	CLEANED_DATA[NDVISTACK == -inf] = np.nan
	CLEANED_DATA[NDVISTACK > 1] = np.nan
	CLEANED_DATA[NDVISTACK < -1] = np.nan

	print ("After Cleaning Missing value is ", np.count_nonzero(np.isnan(CLEANED_DATA)),'which is ' ,percentage(np.count_nonzero(np.isnan(CLEANED_DATA)),NDVISTACK.shape),'%')
	return CLEANED_DATA;

In [ ]:
NDVICLEANEDDATA = cleanNDVI(NDVISTACK)